# Library Import

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from sklearn.neighbors import BallTree
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from typing import Type
import optuna

# Data Load

In [ ]:
# train, test 데이터 불러오기
data_path: str = "~/house/data"
train_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv"))
test_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))
sample_submission: pd.DataFrame = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

In [ ]:
# 금리, 지하철, 학교, 공원 정보 불러오기
interest_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "interestRate.csv"))
subway_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "subwayInfo.csv"))
school_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "schoolinfo.csv"))
park_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "parkInfo.csv"))

# Data Merge

## 1. 금리 데이터 병합
* `interest_data`: 2018년 12월 ~ 2024년 5월까지의 금리
* 계약 연월 기준으로 `interest_data`를 `train_data`로 병합 (2019년 4월 ~ 2023년 12월)
* 계약 연월 기준으로 `interest_data`를 `test_data`로 병합 (2024년 1월 ~ 2024년 6월)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
interest_data

In [ ]:
# 계약 연월 기준으로 interest_data를 train_data로 병합
merged_train = pd.merge(train_data, interest_data, left_on="contract_year_month", right_on="year_month", how="left")
merged_train = merged_train.drop(columns=["year_month"])
merged_train

In [ ]:
# 계약 연월 기준으로 interest_data를 test_data로 병합
merged_test = pd.merge(test_data, interest_data, left_on="contract_year_month", right_on="year_month", how="left")
merged_test = merged_test.drop(columns=["year_month"])
merged_test

In [ ]:
# 금리 결측치 개수 확인 (2024년 6월)
merged_test[merged_test["interest_rate"].isnull()]["contract_year_month"].value_counts()

## 2. 최단거리 데이터 병합

### sklearn의 BallTree를 활용한 haversine 거리 계산 함수

In [ ]:
# 건물-공공장소 사이의 최단거리 장소를 반환하는 함수 정의
def find_closest_distance_haversine(
    train_data: pd.DataFrame, 
    loc_df: pd.DataFrame
) -> pd.DataFrame:
    """건물과 지하철/학교/공원 사이의 최단 haversine 거리와 위치 정보를 반환하는 함수

    Args:
        train_data (pd.DataFrame): 학습(훈련) 또는 테스트 데이터프레임
        loc_df (pd.DataFrame): 위도, 경도를 column으로 갖는 데이터프레임

    Returns:
        pd.DataFrame: index, 위도, 경도, 최단 haversine 거리를 column으로 갖는 데이터프레임
    """
    # degree -> radian 값으로 변환 (for 삼각함수)
    train_coords = np.radians(train_data[["latitude", "longitude"]].values)
    loc_coords = np.radians(loc_df[["latitude", "longitude"]].values)
    
    # Ball Tree 생성 
    tree = BallTree(loc_coords, metric="haversine")

    distances, indices = tree.query(train_coords, k=1) # 가까운 1 지점만 
    distances_meter = distances * 6371000 # 단위를 meter로 변환

    closest_coords = loc_df[["latitude", "longitude"]].iloc[indices.flatten()].values # 가까운 지점 좌표

    # index, 최단거리, 최단거리에 해당하는 지점의 위도, 경도로 이루어진 데이터프레임 생성
    result_df = pd.DataFrame({
        "index" : train_data.index,
        "closest_distance" : distances_meter.flatten(),
        "closest_latitude" : closest_coords[:, 0],
        "closest_longtitude" : closest_coords[:, 1]
    })

    return result_df

### subway 병합

In [ ]:
subway_result = find_closest_distance_haversine(train_data, subway_data)
subway_result.columns = ["index", "nearest_subway_distance", "nearest_subway_latitude", "nearest_subway_longtitude"]
train_data = pd.merge(train_data, subway_result, on="index")

In [ ]:
subway_result = find_closest_distance_haversine(test_data, subway_data)
subway_result.columns = ["index", "nearest_subway_distance", "nearest_subway_latitude", "nearest_subway_longtitude"]
test_data = pd.merge(test_data, subway_result, on="index")

### school 병합

In [ ]:
school_result = find_closest_distance_haversine(train_data, school_data)
school_result.columns = ["index", "nearest_school_distance", "nearest_school_latitude", "nearest_school_longtitude"]
train_data = pd.merge(train_data, school_result, on="index")

In [ ]:
school_result = find_closest_distance_haversine(test_data, school_data)
school_result.columns = ["index", "nearest_school_distance", "nearest_school_latitude", "nearest_school_longtitude"]
test_data = pd.merge(test_data, school_result, on="index")

### park 병합

In [ ]:
park_result = find_closest_distance_haversine(train_data, park_data)
park_result.columns = ["index", "nearest_park_distance", "nearest_park_latitude", "nearest_park_longtitude"]
train_data = pd.merge(train_data, park_result, on="index")

In [ ]:
park_result = find_closest_distance_haversine(test_data, park_data)
park_result.columns = ["index", "nearest_park_distance", "nearest_park_latitude", "nearest_park_longtitude"]
test_data = pd.merge(test_data, park_result, on="index")

## 3. 데이터 병합 결과 확인

In [ ]:
on = merged_train.columns.drop("interest_rate").tolist() # 병합 기준이 될 column 리스트
train_data = pd.merge(merged_train, train_data, on=on, how="left")
# train_data = train_data.drop(columns=["index"])
train_data

In [ ]:
on = merged_test.columns.drop("interest_rate").tolist() # 병합 기준이 될 column 리스트
test_data = pd.merge(merged_test, test_data, on=on, how="left")
# test_data = test_data.drop(columns=["index"])
test_data

# 이상치 탐지

In [ ]:
# 이상치 탐지 함수 정의
def find_outliers(data: pd.Series) -> pd.Series:
    """안 울타리(inner fence) 밖에 있는 데이터(이상치, outlier)를 반환하는 함수

    Args:
        data (pd.Series): 이상치 탐지를 하고싶은 데이터의 column

    Returns:
        pd.Series: 이상치에 해당하는 데이터 Series 반환
    """
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data < lower_bound) | (data > upper_bound)]

# EDA

## 1. 기술통계량

### 1.1 train data

- **`area_m2`**: 면적 (제곱미터)  
- **`contract_year_month`**: 계약 연월  
- **`contract_day`**: 계약일  
- **`contract_type`**: 계약 유형 (0: 신규, 1: 갱신, 2: 모름)  
- **`floor`**: 층수  
- **`built_year`**: 건축 연도  
- **`latitude`**: 위도  
- **`longitude`**: 경도  
- **`age`**: 건물의 나이 (계산된 값)  
- **`deposit`**: 전세 실거래가 (타겟 변수)

- **참고**: 건물 나이(age) = 계약 연도(contract_year) - 건축 연도(built_year): 현 시점의 나이가 아니라 계약 시점의 나이!

In [ ]:
# 건물 나이(age) = 계약 연도(contract_year) - 건축 연도(built_year): 현 시점의 나이가 아니라 계약 시점의 나이!
(train_data["age"] == (train_data["contract_year_month"] // 100) - train_data["built_year"]).value_counts()

In [ ]:
# train_data 결측치 & 데이터 타입 확인
train_data.info()

In [ ]:
# train data 기술통계량 확인
train_data.describe()

#### 1.1.1 column별 종류, 개수 확인

In [ ]:
# 면적 종류, 개수 확인
print("area(m2): ", train_data["area_m2"].sort_values().unique())
print("", len(train_data["area_m2"].sort_values().unique()))

In [ ]:
# 계약 연월 종류, 개수 확인
print(train_data["contract_year_month"].sort_values().unique()) # 201904 ~ 202312
print(len(train_data["contract_year_month"].sort_values().unique()))

In [ ]:
# 계약일 종류, 개수 확인
print(train_data["contract_day"].sort_values().unique())
print(len(train_data["contract_day"].sort_values().unique()))

In [ ]:
# 계약 유형 확인
print(train_data["contract_type"].value_counts())

In [ ]:
# 층수 종류, 개수 확인
print("floor: ", train_data["floor"].sort_values().unique())  # -4층 ~ 0층을 어떻게 처리할지 고민해봐야 한다.
print("# of floor: ", len(train_data["floor"].sort_values().unique())) # -4층 ~ 68층까지 연속적인 층수

In [ ]:
# 층수가 음수인 건물 개수 확인
train_data["floor"].value_counts().sort_index()

In [ ]:
# 층수가 음수인 건물 확인
train_data[(train_data["built_year"] != 2024) & (train_data["floor"] <= 0)]

In [ ]:
# 건축 연도 종류, 개수 확인
print("built year: ", train_data["built_year"].sort_values().unique()) # 1961년을 제외한 1965~2023년까지는 연속적인 연도 / 2024년은 훈련 데이터에서 제거해야 한다.
print("# of built year: ", len(train_data["built_year"].sort_values().unique()))

In [ ]:
# 건물 나이 종류, 개수 확인
print("building age: ", train_data["age"].sort_values().unique()) # -3, -2, -1을 어떻게 처리할지 고민해봐야 한다. (가계약? -> 준공 전 계약)
print("# of building age: ", len(train_data["age"].sort_values().unique()))

In [ ]:
# 전세 실거래가 종류, 개수 확인
print("deposit: ", train_data["deposit"].sort_values().unique()) # -3, -2, -1을 어떻게 처리할지 고민해봐야 한다. (가계약? -> 준공 전 계약)
print("# of deposit: ", len(train_data["deposit"].sort_values().unique())) # 

In [ ]:
# 건축 연도가 2024년인 데이터 수 확인: 총 97개
print(train_data.loc[train_data["built_year"] == 2024])

In [ ]:
# 건축 연도가 2024년인 건물 나이 확인: -3, -1
print("age built in 2024: ", train_data[train_data["built_year"] == 2024]["age"].unique())

In [ ]:
# 건물의 나이가 음수인 건물의 건축 연도 확인 (2024년 제외)
age_year = train_data[(train_data["built_year"] != 2024) & (train_data["age"] < 0)]
age_year

In [ ]:
print("age: ", age_year["age"].sort_values().unique()) # 2024년 외에도 -3, -1 있는지 확인
print("built year: ", age_year["built_year"].sort_values().unique()) # 2024년 이외 건축연도 확인

- 나이가 음수인 건물은 
    - -3: 3개
    - -2: 28개
    - -1: 3323개
- 나이가 음수인 건물의 계약 연도는 2019년 5월 이후
- 나이가 음수인 건물은 건축 연도가 2020년 이후

In [ ]:
age_year["deposit"].describe()

In [ ]:
fig, ax =  plt.subplots(figsize=(12, 5))
sns.boxplot(data=age_year, x="deposit")
plt.show()

In [ ]:
# 가장 큰 이상치 확인
age_year.loc[age_year["deposit"] == max(age_year["deposit"])]

In [ ]:
# 가장 큰 이상치 제거하고 boxplot
fig, ax =  plt.subplots(figsize=(12, 5))
sns.boxplot(x=age_year["deposit"].drop(1344709, axis=0))
plt.show()

In [ ]:
# 건물의 나이가 음수인 건물의 계약, 건축 연도 확인 (2024년 제외)

print("contract year: ", (age_year["contract_year_month"]//100).unique())
print("built year", age_year["built_year"].unique())
print("age: ", age_year["age"].unique())

# age_year[["contract_year_month", "built_year", "age"]]
# grouped = age_year.groupby("age")
# for name, group in grouped:
#     print(name)
#     print(group)

### 1.2 interest data

In [ ]:
interest_data

In [ ]:
# 금리 데이터 결측치 & 데이터 타입 확인
interest_data.info()

In [ ]:
# 금리 데이터 기술통계량 확인
interest_data.describe()

In [ ]:
# 금리 연월 종류, 개수 확인 (나중에 계약 연월과 비교 위해)
print(interest_data["year_month"].sort_values().unique()) # 2018년 12월 ~ 2024년 5월
print(len(interest_data["year_month"].sort_values().unique()))

### 1.3 subway data

In [ ]:
subway_data

In [ ]:
# 지하철 데이터 결측치 & 데이터 타입 확인
subway_data.info()

In [ ]:
# 지하철 데이터 기술통계량 확인
subway_data.describe()

### 1.4 school data

In [ ]:
school_data

In [ ]:
# 학교 데이터 결측치 & 데이터 타입 확인
school_data.info()

In [ ]:
# 학교 데이터 기술통계량 확인
school_data.describe(include="all")

In [ ]:
# 학교 종류별 개수 확인
print(school_data["schoolLevel"].value_counts())
print(school_data["schoolLevel"].value_counts(normalize=True))

### 1.5 park data

In [ ]:
park_data

In [ ]:
# 공원 데이터 결측치 & 데이터 타입 확인
park_data.info()

In [ ]:
# 공원 데이터 기술통계량 확인
park_data.describe()

## 2. 데이터 시각화

### [Univariate]
### 2.1 Box Plot

#### 2.1.1 train data

In [ ]:
# train data 상자 그림 시각화 (인덱스, 계약 유형 제외)
fig, axes =  plt.subplots(1, train_data.drop(columns=["index", "contract_type"]).shape[1], figsize=(30, 7))

for idx, col in enumerate(train_data.drop(columns=["index", "contract_type"])):
    sns.boxplot(data=train_data.drop(columns=["index", "contract_type"]), y=col, ax=axes[idx])

    axes[idx].set_title(col)
    axes[idx].set_ylabel("")

plt.show()

만약 계약 유형별 feature들의 상자 그림이 궁금하다면?

In [ ]:
# train data 상자 그림 시각화 (인덱스, 계약 유형 제외)
fig, axes =  plt.subplots(1, train_data.drop(columns=["index", "contract_type"]).shape[1], figsize=(30, 7))

for idx, col in enumerate(train_data.drop(columns=["index", "contract_type"])):
    sns.boxplot(data=train_data.drop(columns=["index", "contract_type"]), y=col, ax=axes[idx],
                hue=train_data["contract_type"]
                )

    axes[idx].set_title(col)
    axes[idx].set_ylabel("")

plt.show()

In [ ]:
sns.boxplot(data=train_data, x="contract_type", y="deposit", hue="contract_type")
plt.show()

#### 2.1.2 interest data

In [ ]:
# 금리 상자 그림 시각화 (금리 연월 제외)
sns.boxplot(data=interest_data, y="interest_rate")
plt.show()

#### 2.1.3 subway data

In [ ]:
# 지하철 위도, 경도 상자 그림 시각화
fig, axes =  plt.subplots(1, 2, figsize=(12, 5))
sns.boxplot(data=subway_data, y="latitude", ax=axes[0])
sns.boxplot(data=subway_data, y="longitude", ax=axes[1])
plt.show()

#### 2.1.4 school data

In [ ]:
# 학교 위도, 경도 상자 그림 시각화
fig, axes =  plt.subplots(1, 2, figsize=(12, 5))
sns.boxplot(data=school_data, y="latitude", ax=axes[0])
sns.boxplot(data=school_data, y="longitude", ax=axes[1])
plt.show()

In [ ]:
# 학교 레벨에 따른 위도, 경도 상자 그림 시각화
fig, axes =  plt.subplots(1, 2, figsize=(12, 5))
sns.boxplot(school_data, x="schoolLevel", y="latitude", ax=axes[0])
sns.boxplot(school_data, x="schoolLevel", y="longitude", ax=axes[1])
plt.show()

#### 2.1.5 park data

In [ ]:
# 공원 위도, 경도, 면적 상자 그림 시각화 (금리 연월 제외)
fig, axes =  plt.subplots(1, 3, figsize=(18, 5))
sns.boxplot(data=park_data, y="latitude", ax=axes[0])
sns.boxplot(data=park_data, y="longitude", ax=axes[1])
sns.boxplot(data=park_data, y="area", ax=axes[2])
plt.show()

### 2.2 Histogram / Count Plot

#### 2.2.1 train data

##### 면적 분포

In [ ]:
# 면적 히스토그램 시각화
fig, ax =  plt.subplots(figsize=(15, 5))
sns.histplot(data=train_data, x="area_m2", bins=np.arange(0, 310, 10))
plt.show()

##### 계약 연/월/일 분포

In [ ]:
# 계약 연/월/일별 히스토그램 시각화
fig, axes =  plt.subplots(3, 1, figsize=(10, 12))

sns.histplot(x=(train_data["contract_year_month"] // 100).sort_values().astype(str), ax=axes[0])
sns.histplot(x=(train_data["contract_year_month"] % 100).sort_values().astype(str), ax=axes[1])
sns.histplot(x=train_data["contract_day"].sort_values().astype(str), ax=axes[2])
plt.show()

##### 계약 유형 분포

In [ ]:
print(train_data["contract_type"].value_counts())
print(train_data["contract_type"].value_counts(normalize=True))

In [ ]:
# 계약 유형별 빈도수 시각화
sns.countplot(data=train_data, x="contract_type", hue="contract_type")
plt.show()

##### 층수 분포

In [ ]:
# 층수 히스토그램 시각화
fig, ax =  plt.subplots(figsize=(20, 5))
sns.histplot(train_data["floor"].sort_values().astype(str))#, kde=True)
plt.show()

##### 건축 연도 분포

In [ ]:
# 건축 연도 히스토그램 시각화
fig, ax =  plt.subplots(figsize=(20, 5))
sns.histplot((train_data["built_year"].sort_values() % 100).astype(str))#, kde=True)
plt.show()

##### 위도, 경도 분포

In [ ]:
# 위도, 경도 히스토그램 시각화
fig, axes =  plt.subplots(2, 1, figsize=(15, 10))
sns.histplot(train_data["latitude"], kde=True, bins=100, ax=axes[0])
sns.histplot(train_data["longitude"], kde=True, bins=100, ax=axes[1])
plt.show()

##### 건물 나이 분포

In [ ]:
# 건물 나이 히스토그램 시각화
fig, ax =  plt.subplots(figsize=(20, 5))
sns.histplot(train_data["age"].sort_values().astype(str), kde=True)
plt.show()

##### 전세 실거래가 분포

In [ ]:
# 전세 실거래가 히스토그램 시각화
fig, ax = plt.subplots(figsize=(20, 5))
sns.histplot(data=train_data, x="deposit", kde=True)
plt.show()

#### 2.2.2 interest data

In [ ]:
# 금리 히스토그램 시각화
sns.histplot(data=interest_data, x="interest_rate", bins=np.arange(0.5, 4.5, 0.1))
plt.show()

#### 2.2.3 subway data

In [ ]:
# 학교 위도, 경도 히스토그램 시각화
fig, axes =  plt.subplots(2, 1, figsize=(15, 10))
sns.histplot(data=subway_data, x="latitude", kde=True, ax=axes[0])
sns.histplot(data=subway_data, x="longitude", kde=True, ax=axes[1])
plt.show()

#### 2.2.4 school data

In [ ]:
# 학교 레벨별 위도, 경도 히스토그램 시각화
fig, axes =  plt.subplots(2, 1, figsize=(15, 10))
sns.histplot(data=school_data, x="latitude", hue="schoolLevel", multiple="stack", kde=True, ax=axes[0])
sns.histplot(data=school_data, x="longitude", hue="schoolLevel", multiple="stack", kde=True, ax=axes[1])
plt.show()

#### 2.2.5 park data

In [ ]:
# 공원 위도, 경도 히스토그램 시각화
fig, axes =  plt.subplots(2, 1, figsize=(15, 10))
sns.histplot(data=park_data, x="latitude", kde=True, ax=axes[0])
sns.histplot(data=park_data, x="longitude", kde=True, ax=axes[1])
plt.show()

In [ ]:
# 공원 면적 히스토그램 시각화
# fig, axes =  plt.subplots(figsize=(15, 5))
sns.histplot(data=park_data, x="area")
plt.show()

### [Bivariate]
### 2.3 Scatter Plot

In [ ]:
# # train data 산점도 시각화 (인덱스 제외)
# fig, axes =  plt.subplots(3, 3, figsize=(15, 15))

# for idx, col in enumerate(train_data.drop(columns=["index", "contract_type"])):
#     sns.scatterplot(data=train_data.drop(columns=["index", "contract_type"]), x=col, y="deposit",
#             #    hue=train_data["contract_type"], # 계약 유형별 산점도가 보고싶은 경우
#                ax=axes[idx//3, idx%3])
    
#     if idx // 3 != 0:
#         axes[idx//3, idx%3].set_ylabel("")

# plt.show()

### 2.4 Lineplot

#### (계약 연월 순) 전세가, 금리 선 그래프 시각화

In [ ]:
# 계약연월 int -> date 타입으로 변환
train_data_copy = train_data.copy()
interest_data_copy = interest_data.copy()

train_data_copy["contract_year_month"] = pd.to_datetime(train_data_copy["contract_year_month"], format="%Y%m")
interest_data_copy["year_month"] = pd.to_datetime(interest_data_copy["year_month"], format="%Y%m")

# 계약 연월별 평균 전세가, 금리 데이터프레임 생성
monthly_avg_deposit = train_data_copy.groupby("contract_year_month").agg({"deposit": "mean"}).reset_index()
monthly_avg_interest = interest_data_copy.groupby("year_month").agg({"interest_rate": "mean"}).reset_index()

# 계약 연월 기준으로 두 데이터프레임 병합
monthly_avg = monthly_avg_deposit.merge(monthly_avg_interest, left_on="contract_year_month", right_on="year_month", how="left")
monthly_avg.drop(columns=['year_month'], inplace=True)
monthly_avg.head()

In [ ]:
# 계약 연월에 따른 (평균) 전세가, 금리 시계열 데이터 시각화
fig, ax =  plt.subplots(figsize=(12, 5))
sns.lineplot(data=monthly_avg, x="contract_year_month", y="deposit", label="deposit")
plt.legend(loc="upper center")
plt.twinx()
sns.lineplot(data=monthly_avg, x="contract_year_month", y="interest_rate", color="orange", label="interest_rate")

plt.legend(loc="lower center")
plt.show()

### 2.5 Heatmap

In [ ]:
# train data 변수별 상관행렬(히트맵) 시각화
fig, ax = plt.subplots(figsize=(10, 9))
sns.heatmap(train_data.drop(columns="index").corr(), ax=ax,
            vmin=-1, vmax=1, center=0,
            cmap="coolwarm",
            annot=True, fmt=".2f",
            linewidth=0.1,
           )
plt.show()

## 3. Clustering (미완)

# Feature Engineering

In [ ]:
# 공공장소 최단거리-전세가 산점도 시각화 (train data)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
sns.scatterplot(x="nearest_subway_distance", y="deposit", data=train_data, ax=axes[0])
sns.scatterplot(x="nearest_school_distance", y="deposit", data=train_data, ax=axes[1])
sns.scatterplot(x="nearest_park_distance", y="deposit", data=train_data, ax=axes[2])
plt.show()

In [ ]:
# 공공장소 최단거리 히스토그램 시각화 (train data)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
sns.histplot(x="nearest_subway_distance", data=train_data, ax=axes[0])
sns.histplot(x="nearest_school_distance", data=train_data, ax=axes[1])
sns.histplot(x="nearest_park_distance", data=train_data, ax=axes[2])
plt.show()

In [ ]:
# 공공장소 최단거리 히스토그램 시각화 (test data)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
sns.histplot(x="nearest_subway_distance", data=test_data, ax=axes[0])
sns.histplot(x="nearest_school_distance", data=test_data, ax=axes[1])
sns.histplot(x="nearest_park_distance", data=test_data, ax=axes[2])
plt.show()

In [ ]:
# 건축 연도 2024년, 음수 층수 데이터 제거 (train data)
train_data = train_data[(train_data["built_year"] < 2024) & (train_data["floor"] >= 0)]

In [ ]:
# 날짜 데이터 변환
# train_data["contract_year_month"] = pd.to_datetime(train_data["contract_year_month"], format="%Y%m")
# train_data["built_year"] = pd.to_datetime(train_data["built_year"], format="%Y")

# test_data["contract_year_month"] = pd.to_datetime(test_data["contract_year_month"], format="%Y%m")
# test_data["built_year"] = pd.to_datetime(test_data["built_year"], format="%Y")

In [ ]:
# 면적, 층수, 전세가 로그 변환 (train data)
train_data["log_area_m2"] = np.log1p(train_data["area_m2"])
train_data["log_floor"] = np.log1p(train_data["floor"])
train_data["log_deposit"] = np.log1p(train_data["deposit"])
# 면적, 층수 로그 변환 (test data)
test_data["log_area_m2"] = np.log1p(test_data["area_m2"])
test_data["log_floor"] = np.log1p(test_data["floor"])

# 거리 로그 변환 (train data)
train_data["log_subway_distance"] = np.log1p(train_data["nearest_subway_distance"])
train_data["log_school_distance"] = np.log1p(train_data["nearest_school_distance"])
train_data["log_park_distance"] = np.log1p(train_data["nearest_park_distance"])
# 거리 로그 변환 (test data)
test_data["log_subway_distance"] = np.log1p(test_data["nearest_subway_distance"])
test_data["log_school_distance"] = np.log1p(test_data["nearest_school_distance"])
test_data["log_park_distance"] = np.log1p(test_data["nearest_park_distance"])

In [ ]:
# 학습에 사용할 feautre 선택
cols = ["log_area_m2", "log_floor", "log_subway_distance", "log_school_distance", "log_park_distance", "contract_year_month", "built_year", "interest_rate", "latitude", "longitude"]
X_train = train_data[cols]
y_train = train_data["log_deposit"]
X_test = test_data[cols]

# Model Training

In [ ]:
# 모델 객체 생성
model = XGBRegressor(tree_method="hist", device="cuda", random_state=42)
# model.get_params()
model.fit(X_train, y_train)

## Cross Validation

In [ ]:
# k-fold 교차 검증 객체 생성
kfold = KFold(n_splits=5)

# k-fold 교차 검증으로 모델 예측 및 평가
score = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
print(f"average of MAE for k-fold CV = {-score.mean():.4f}")

## Hyperparameter Tuning

In [ ]:
# 목적함수 정의
def objective(trial: Type[optuna.trial.Trial]) -> float:
    """하이퍼파라미터셋 탐색을 최적화할 목적함수

    Args:
        trial (Type[optuna.trial.Trial]): optuna.trial.Trial 클래스

    Returns:
        float: 설정한 평가지표 (현재는 MAE)
    """
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "tree_method": "hist",
        "device": "cuda",
        "random_state": 42
    }
    model = XGBRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=kfold, scoring="neg_mean_absolute_error")
    return -score.mean()

In [ ]:
# optuna로 튜닝한 하이퍼파라미터셋, 평가지표 탐색
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
best_params = study.best_params
print("Best parameters for XGBoost: ", best_params)

# Inference

In [ ]:
# 모델 추론
best_model = XGBRegressor(
    **best_params,
    # tree_method="hist",
    # device="cuda", # 현재 두 옵션 활성화하면 predict에서 error 발생
    random_state=42
)
best_model.fit(X_train, y_train)
y_pred_log = best_model.predict(X_test)
y_pred = np.expm1(y_pred_log) # 지수변환 (로그변환의 역변환)

In [ ]:
# 지수변환 전/후 예측값 히스토그램 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(y_pred_log, ax=axes[0])
sns.histplot(y_pred, ax=axes[1])
plt.show()

# Outut Files Save

In [ ]:
sample_submission["deposit"] = y_pred
sample_submission.to_csv("output.csv", index=False)